In [1]:
import h5py
import numpy as np
from astropy.table import Table
from astropy.cosmology import Planck18
import astropy.units as u

In [2]:
def read_DCO(var = 'v02.35.02/N1e7_Fiducial_AllDCO_AIS/',
             rate_key = 'Rates_mu00.025_muz-0.049_alpha-1.79_sigma01.129_sigmaz0.048_a0.017_b1.487_c4.442_d5.886_zBinned', 
             redshift_interest  = 0.2):
    
    ####################################
    data_dir = '/mnt/ceph/users/lvanson/CompasOutput/' + var

    # Read the rates data
    RateFile = h5py.File(data_dir+'/CosmicIntegration/Rate_info.h5', 'r')    

    # Print the keys in the simulation data
    print(RateFile[rate_key].keys())

    # Print the shape of each dataset in the simulation data
    for key in RateFile[rate_key].keys():
        print(np.shape(RateFile[rate_key][key][()]))

    ####################################
    # Read COMPAS simulation data
    dataFile = h5py.File(data_dir+'/MainRun/COMPAS_Output_wWeights.h5', 'r')

    # Set the key for the double compact object (DCO) data
    DCO_key = 'BSE_Double_Compact_Objects'
    # Set the list of keys to extract from the DCO data
    key_list = ['SEED', 'Mass(1)','Mass(2)','mixture_weight', 'Stellar_Type(1)', 'Stellar_Type(2)', 'Merges_Hubble_Time'] #'Metallicity@ZAMS(1)', 'Immediate_RLOF>CE', 'Optimistic_CE',

    # Extract DCO data to astropy table
    DCO = Table()
    for key in key_list:
        DCO[key] = dataFile[DCO_key][key][()]
    # Calculate the more massive and less massive component masses and chirp mass
    DCO['M_moreMassive'] = np.maximum(dataFile[DCO_key]['Mass(1)'][()], dataFile[DCO_key]['Mass(2)'][()])
    DCO['M_lessMassive'] = np.minimum(dataFile[DCO_key]['Mass(1)'][()], dataFile[DCO_key]['Mass(2)'][()])
    
    # Reduce the DCO table to only NSNS that merge in a Hubble time
    DCO_merger_calculated = DCO[RateFile[rate_key]['DCOmask'][()]] #RateFile[rate_key]['DCOmask'][()] reduces BSE_Double_Compact_Objects to the same shape as RateFile

    NSNS_bool = np.logical_and(DCO_merger_calculated['Stellar_Type(1)'] == 13, DCO_merger_calculated['Stellar_Type(2)'] == 13)
    
    print(f'You might not have always calculated all rates! sum NSNS_bool = {sum(NSNS_bool)}')

    merger_bool = DCO_merger_calculated['Merges_Hubble_Time'] == True


    NSNStable  = DCO_merger_calculated[np.logical_and(NSNS_bool, merger_bool)]
    rates_NSNS = RateFile[rate_key]['merger_rate'][np.logical_and(NSNS_bool, merger_bool)]

    # Extract redshift for the current frame
    redshifts = RateFile[rate_key]['redshifts'][()]
    # Get the redshift index closest to the redshift of interest
    redshift_i = np.argmin(np.abs(redshifts - redshift_interest))

    return NSNStable, rates_NSNS, redshifts, redshift_i

In [3]:
NSNStable, rates_NSNS, redshifts, redshift_i = read_DCO()

<KeysViewHDF5 ['DCOmask', 'SEED', 'merger_rate', 'redshifts']>
(2394444,)
(1757148,)
(1757148, 100)
(100,)
You might not have always calculated all rates! sum NSNS_bool = 21595


In [4]:
m1_inj = np.array(NSNStable['M_moreMassive'])
m2_inj = np.array(NSNStable['M_lessMassive'])
rate_inj = rates_NSNS
z_inj = redshifts

z_inj+=0.001

# Absolute Rate

In [5]:
Mc_inj = (m1_inj*m2_inj)**0.6/(m1_inj+m2_inj)**0.2
logq_inj = np.log(m2_inj/m1_inj)

# in a year
dNdTobs_grid = rate_inj.T*(4*np.pi*Planck18.differential_comoving_volume(z_inj[:,None]).to(u.Gpc**3/u.sr).value/(1+z_inj[:,None]))
dNdTobs = np.sum(dNdTobs_grid)

# Tobs
Tobs = np.linspace(0.1,30,10)
Nobs_arr = np.random.poisson(dNdTobs*Tobs/(30*12))
print(Nobs_arr)

[  108  3595  6897 10552 13741 17033 20525 23946 27321 30483]


In [6]:
with h5py.File("Nobs.h5", "w") as file:
    file.create_dataset('Nobs_arr', data=Nobs_arr)
file.close()